Imports and Setup

In [44]:
import socket
import subprocess
import platform
import datetime
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict

!apt-get install -y iputils-ping

# Display settings for Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
iputils-ping is already the newest version (3:20211215-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


Configuration

Define the targets you want to scan. You can modify the TARGETS list and the PORTS_TO_SCAN dictionary based on your server architecture.

In [45]:
# List of servers to check (IPs or Hostnames)
TARGETS = [
    'google.com',
    '127.0.0.1',   # Localhost
    '8.8.8.8',     # Google DNS
    'scanme.nmap.org' # Permitted scan target for testing
]

# Common ports to check.
# Format: {Port Number: 'Service Name'}
PORTS_TO_SCAN = {
    21: 'FTP',
    22: 'SSH',
    80: 'HTTP',
    443: 'HTTPS',
    3306: 'MySQL',
    3389: 'RDP',
    8080: 'Alt-HTTP'
}

# Socket timeout in seconds (Lower is faster, but might miss lagging servers)
TIMEOUT = 1.0

print(f"Targets Configured: {len(TARGETS)}")
print(f"Ports per Target: {len(PORTS_TO_SCAN)}")

Targets Configured: 4
Ports per Target: 7


Connectivity Checker (Ping)

This function handles the ICMP request. It automatically detects the operating system to send the correct ping command parameters.

In [46]:
def check_ping(host: str) -> bool:
    """
    Pings a host to check basic reachability.
    Returns True if reachable, False otherwise.
    """
    # specific parameters for Windows vs Linux/Mac
    param = '-n' if platform.system().lower() == 'windows' else '-c'

    # Building the command: ping -c 1 -w 1000 host
    command = ['ping', param, '1', host]

    try:
        # We pipe output to DEVNULL to keep the console clean
        return subprocess.call(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) == 0
    except Exception as e:
        print(f"Error pinging {host}: {e}")
        return False

TCP Port Scanner

This function attempts to establish a TCP connection. If it succeeds (returns 0), the port is open.

In [47]:
def scan_port(host: str, port: int) -> dict:
    """
    Checks if a specific TCP port is open on a host.
    """
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(TIMEOUT)

    result = {
        'host': host,
        'port': port,
        'service': PORTS_TO_SCAN.get(port, 'Unknown'),
        'status': 'CLOSED'
    }

    try:
        # connect_ex returns 0 on success
        if sock.connect_ex((host, port)) == 0:
            result['status'] = 'OPEN'
    except Exception:
        result['status'] = 'ERROR'
    finally:
        sock.close()

    return result

def scan_host_ports(host: str) -> List[dict]:
    """
    Scans all configured ports for a single host using threading.
    """
    results = []
    # ThreadPoolExecutor allows scanning multiple ports simultaneously
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scan_port, host, port) for port in PORTS_TO_SCAN.keys()]
        for future in futures:
            results.append(future.result())
    return results

Main Orchestrator

This cell combines the ping check and the port scan into a single report.

In [48]:
def run_network_diagnostic(targets: List[str]):
    """
    Runs the full diagnostic suite on a list of targets.
    """
    print(f"🚀 Starting Network Diagnostic at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    report_data = []

    for host in targets:
        print(f"Checking host: {host}...", end=" ")

        # 1. Check Connectivity
        is_alive = check_ping(host)
        status_icon = "🟢" if is_alive else "🔴"
        print(f"{status_icon} (Ping: {'Success' if is_alive else 'Failed'})")

        # 2. Check Ports (Only if host is alive or if you want to force check)
        if is_alive:
            port_results = scan_host_ports(host)
            for res in port_results:
                report_data.append({
                    'Timestamp': datetime.datetime.now(),
                    'Target': host,
                    'Ping_Status': 'Online',
                    'Port': res['port'],
                    'Service': res['service'],
                    'Port_Status': res['status']
                })
        else:
            # Log the failure even if we can't scan ports
            report_data.append({
                'Timestamp': datetime.datetime.now(),
                'Target': host,
                'Ping_Status': 'Offline',
                'Port': 'N/A',
                'Service': 'N/A',
                'Port_Status': 'Unreachable'
            })

    # Convert to Pandas DataFrame for nice formatting
    df = pd.DataFrame(report_data)
    return df

# Run the tool
health_report = run_network_diagnostic(TARGETS)

🚀 Starting Network Diagnostic at 2026-01-20 16:08:12

Checking host: google.com... 🟢 (Ping: Success)
Checking host: 127.0.0.1... 🟢 (Ping: Success)
Checking host: 8.8.8.8... 🟢 (Ping: Success)
Checking host: scanme.nmap.org... 🟢 (Ping: Success)


Report and Visualization

Finally, we display the results. We separate "Open" ports from "Closed" or "Offline" instances to highlight risks or failures.

In [49]:
# Display Full Report
print("\n FULL DIAGNOSTIC REPORT ")
display(health_report)

# Filter for OPEN ports (What services are exposed?)
print("\n EXPOSED SERVICES (OPEN PORTS) ")
open_ports = health_report[health_report['Port_Status'] == 'OPEN']
if not open_ports.empty:
    display(open_ports[['Target', 'Port', 'Service']])
else:
    print("No open ports found on reachable hosts.")

# Filter for OFFLINE hosts
print("\n OFFLINE HOSTS ")
offline_hosts = health_report[health_report['Ping_Status'] == 'Offline']['Target'].unique()
if len(offline_hosts) > 0:
    for host in offline_hosts:
        print(f"❌ {host} is unreachable.")
else:
    print("✅ All targets are responding to Ping.")


 FULL DIAGNOSTIC REPORT 


,Timestamp,Target,Ping_Status,Port,Service,Port_Status
0,2026-01-20 16:08:13.190511,google.com,Online,21,FTP,CLOSED
1,2026-01-20 16:08:13.190522,google.com,Online,22,SSH,CLOSED
2,2026-01-20 16:08:13.190524,google.com,Online,80,HTTP,OPEN
3,2026-01-20 16:08:13.190526,google.com,Online,443,HTTPS,OPEN
4,2026-01-20 16:08:13.190528,google.com,Online,3306,MySQL,CLOSED
5,2026-01-20 16:08:13.190529,google.com,Online,3389,RDP,CLOSED
6,2026-01-20 16:08:13.190531,google.com,Online,8080,Alt-HTTP,CLOSED
7,2026-01-20 16:08:13.195412,127.0.0.1,Online,21,FTP,CLOSED
8,2026-01-20 16:08:13.195419,127.0.0.1,Online,22,SSH,CLOSED
9,2026-01-20 16:08:13.195420,127.0.0.1,Online,80,HTTP,CLOSED



 EXPOSED SERVICES (OPEN PORTS) 


,Target,Port,Service
2,google.com,80,HTTP
3,google.com,443,HTTPS
13,127.0.0.1,8080,Alt-HTTP
17,8.8.8.8,443,HTTPS
22,scanme.nmap.org,22,SSH
23,scanme.nmap.org,80,HTTP



 OFFLINE HOSTS 
✅ All targets are responding to Ping.
